# Baseline RAG example

This is a simple example of a baseline RAG application which purpose is to answer questions about the fantasy series [Malazan Universe](https://malazan.fandom.com/wiki/Malazan_Wiki) created by Steven Erikson and Ian C. Esslemont.

First the example will show each step of a baseline RAG pipeline including **Indexing**, **Retrieval** and **Generation**. This is done in order to show the architecture without the abstraction provided by frameworks like LlamaIndex and LangChain.
Then a more "normal" example will be shown using LlamaIndex.

As a vector database, we will use [ChromaDB](https://docs.trychroma.com/), but this can easily be exchanged with other databases.

In this example, we will use the following technologies

- OpenAI API
- ChromaDB
- LlamaIndex


### Setup libraries and environment


In [1]:
#%pip install chromadb llama-index-vector-stores-chroma

In [23]:
import os

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb import Settings
from IPython.display import Markdown, display
from llama_index.core import PromptTemplate, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from openai import OpenAI, AzureOpenAI

import importlib
import util

# Suppose you made some changes to util.py

#importlib.reload(util.helpers)
from util.helpers import create_and_save_md_files, get_malazan_pages, get_theoffice_pages

### Environment variables

For this example you need to use an OpenAI API key. Go to [your API keys](https://platform.openai.com/api-keys) in the OpenAI console to generate one.

Then add the following to a `.env` file in the root of the project.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```


In [54]:
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai_client = AzureOpenAI(
    api_key=OPENAI_API_KEY,  
    api_version="2024-05-01-preview", # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [55]:

#openai_client = AzureOpenAI(api_key=OPENAI_API_KEY)
#openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#    api_key=OPENAI_API_KEY,
#    model_name="text-embedding-3-small"
#)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-ada-002",
    api_type="azure"
)

chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag-pdf-docs/chromadb", settings=Settings(allow_reset=True))

## Indexing

In this step, we will index the documents in our vector database. This will allow us to retrieve the most relevant documents when we ask a question.

We will use ChromaDB as our vector database and 'text-embedding-3-small' from OpenAI as our embedding model.


#### Fetch and process saved documents

First we need to fetch the documents we saved earlier.

Then we will process the documents in order to add them to our vector database.
The `SimpleDirectoryReader` fetches each section of the markdown file
Then each section is split in to smaller chunks of text and each chunk is embedded using the OpenAI API.


In [56]:
documents = SimpleDirectoryReader('./data/docs').load_data()
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=20)

document_data = []

for document in documents:
    chunks = text_splitter.split_text(document.text)
    for idx, chunk in enumerate(chunks):
        embedding = openai_client.embeddings.create(
            input=chunk, model="text-embedding-ada-002")
        document_data.append({
            "id": f"{document.id_}-{idx}",
            "text": chunk,
            "metadata": document.metadata,
            "embedding": embedding.data[0].embedding
        })

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)
Ignoring wrong pointing object 96 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)


In [27]:
print("embeddings dim", len(document_data[0]['embedding']))
print(document_data[0])

embeddings dim 1536
{'id': '040ca88d-853b-4d62-8440-67cf5a27cfeb-0', 'text': 'Bernard Parker, left, was rated high  risk;  Dylan  Fugett  was rated low risk. (Josh Ritchie for \nProPublica)\nMachine  Bias\nThere’s software used across  the country to predict future criminals.\nAnd it’s biased against blacks.\nby Julia  Angwin,  Jeﬀ Larson,  Surya Mattu and Lauren  Kirchner,  ProPublica\nMay 23, 2016\nON A SPRING  AFTERNOON  IN 2014,  Brisha  Borden  was running  late to pick up her god-sister  \nfrom school  when  she spotted  an unlocked  kid’s  blue  Huﬀy bicycle  and a silver Razor scooter. \nBorden  and a friend  grabbed  the bike and scooter and tried to ride them down  the street  in the \nFort Lauderdale  suburb  of Coral Springs.\n1Julia  Angwin et al. (May 23, 2016).  Machine Bias. url:\nhttps://www.propublica.org/article/machine-bias-\nrisk-assessments-in-criminal-sentencing (visited on\n04/04/2018)⇤\n⇤Online-artiklen indeholder også links til andre artilker og til bagvedligg

#### Add documents to ChromaDB


In [57]:
documents = [doc["text"] for doc in document_data]
embeddings = [doc["embedding"] for doc in document_data]
metadatas = [doc["metadata"] for doc in document_data]
ids = [doc["id"] for doc in document_data]

In [58]:
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="vtdat", metadata={"hnsw:space": "cosine"}, embedding_function=openai_ef)

In [59]:
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids)

## Retrieval

In this step, we will retrieve the most relevant documents to a given question. We will use the vector database to retrieve the most similar documents to the question.

In order to do this we will use the `text-embedding-3-small` model (**the same model used to index the documents**) from OpenAI to embed the question and then use the vector database to retrieve the most similar documents.

We will retrieve the top 5 documents based on the _cosine similarity_ between the question and the documents. Other similarity metrics can be used as well like squared L2 or inner product.

Change `cosine` to `l2` or `ip` when creating the collection above to try these out.


In [40]:
queries = [#"Hvad står P, L, A, C og E for i PLACE-akronymet af John Ziman",
         "Redegør for hovedtrækkene i sagen om COMPAS-algoritmen som den beskrives i Angwin et al. (2016)."
         ]

query = queries[-1]

In [41]:
result = collection.query(query_texts=[query], n_results=5)
context = result["documents"][0]
#display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

formatted_text = "\n\n------------\n\n".join(context)

# Display the formatted markdown
display(Markdown(f"{formatted_text}"))

En
algoritme som COMP AS, der har en større tilbøjelighed til at holde farvede i fængsel end
ikke-farvede, vil bidrage til fastholde denne strukturelle ulighed. En datadreven algoritme
afspejler virkeligheden, som den er, men hvis virkeligheden er racistisk, vil algoritmen også
blive det, og hvis algoritmen bliver brugt uden forbehold, kan den bidrage til at fastholde 685
historisk betingede diskriminerende strukturer. Der er derimod ikke noget, der tyder på, at
mænds kortere levealder er et udtryk for strukturel diskrimination. Så med andre ord er
det vigtigt, at se på den kontekst, en algoritme skal indgå i, når man overvejer, om den er
retfærdig.
Pandoras black box 690
I en traditionel statistisk model vil man have nogenlunde styr på de variable, modellen
inddrager. Det er dog værd at bemærke, at tingene selv med traditionelle algoritmer ik-
ke altid er så klare; COMP AS-algoritmen inddrager således ikke direkte etnicitet som en
variabel. Den inddrager til gengæld flere variable, der er tæt korrelerede med etnicitet, og
dermed indgår etnicitet indirekte i algoritmen. Det gør naturligvis tingene mere besværlige, 695
16Henrik Kragh Sørensen og Mikkel Willum Johansen (apr. 2022). „Invitation til de datalogiske fags
videnskabsteori“ . Lærebog til brug for undervisning ved Institut for Naturfagenes Didaktik, Københavns
Universitet. Under udarbejdelse.
Kapitel 10, version 151 (2021-06-05).

------------

Algoritmen til forudsigelse af frafald inddrog således direkte elevernes etnicitet
som en faktor i beregningen, men hvad nu hvis algoritmen havde en bias så den systematisk
gav elever med en etnicitet en højre risikovurdering end elever med en anden?
Spørgsmålet om diskrimination i algoritmer er navnligt blevet diskuteret i forbindelse 595
med den såkaldte COMP AS-algoritme, der bruges i det amerikanske retsvæsen til vurdering
af kriminelles tilbagefaldsrisiko. En gennemgang af algoritmen viste imidlertid en systema-
tisk skævhed i de fejl, algoritmen lavede. Således havde farvede, der ikke senere begik ny
kriminalitet, en langt højere chance for fejlagtigt at få en høj risikoscore end hvide, og hvide
havde omvendt en højere chance for fejlagtigt at få en lav score end farvede (Angwin m.fl., 600
2016 ).
Hvis man imidlertid tager udgangspunkt i de to kategorier høj- og lav-risiko, og under-
14Henrik Kragh Sørensen og Mikkel Willum Johansen (apr. 2022). „Invitation til de datalogiske fags
videnskabsteori“ . Lærebog til brug for undervisning ved Institut for Naturfagenes Didaktik, Københavns
Universitet. Under udarbejdelse.
Kapitel 10, version 151 (2021-06-05).

------------

Det er ret ukontroversielt. Det er straks mere kontroversielt, at man 575
bl.a. i det amerikanske retsvæsen siden starten af den 20. århundrede har brugt lignende
beregninger til at vurdere kriminelles risiko for at begå ny kriminalitet – og at den type
beregninger er blevet brugt i vurderingen strafudmåling, prøveløsladelser mv. (Carlson, A.
(2017). The Need for T ransparency in the Age of Predictive Sentencing Algorithms. Iowa
Law Review, 103(1), 303–329.). I de sidste årtier har lettere adgang til store datamængder 580
og fremkomsten af diverse machine learning teknikker gjort det lettere at udvikle den type
systemer og diverse former for prædiktive algoritmer bruges i dag på en lang række områder
fra reklamer til sortering af jobansøgninger.
Det er dog ikke helt uproblematisk at bruge prædiktive algoritmer. Som vi så tidligere, er
der en række epistemiske problemer forbundet med machine learning. Der er typisk en meget 585
direkte sammenhæng mellem epistemiske og etiske overvejelser; hvis de etiske overvejelser
inddrager utilititaristiske komponenter er det i høj grad væsentligt at forstå, hvor godt en
given algoritme i praksis virker. Ville den etiske vurdering af algoritmen til forudsigelse af
frafald i gymnasiet fx falde anderledes ud, hvis systemet kun havde en træfsikkerhed på
67%? Der er imidlertid også et anden og vanskeligere etisk aspekt specielt hvis man bruger 590
algoritmiske forudsigelser til at træffe afgørelser, der har betydning for enkeltindividers
muligheder.

------------

F or det første er de etiske argumenter
for at bruge prædiktive algoritmer typisk utilitaristiske; den grundlæggende påstand er, at
en algoritme som COMP AS på den eller anden måde øger den samlede nytte i samfundet og
det er på den baggrund algoritmen skal vurderes (der kan selvfølgelig også være ikke-etiske
begrundelser som kommercielle interesser eller andet). 620
F or andet skal være klar over, at prædiktive algoritmer altid diskriminerer, i den forstand
at din fremtidige adfærd estimeres på baggrund af den fortidige adfærd af den gruppe, som
du anses for at tilhøre. Hvis man fx vil estimere hvor længe en 25 årig, ikke-rygende, køben-
havnsk studerende lever, vil man se på, hvor længe folk i en referenceklasse af personer, der
minder om vores 25-årige studerende i gennemsnit har levet. Det er ganske simpelt sådan, 625
statistiske forudsigelser fungerer (i det mindste i den frekventistisk tolkning af sandsynlig-
hed, som er mest naturlig i denne sammenhæng). Spørgsmålet er altså ikke om algoritmerne
må diskriminere, men i højere grad hvilke faktorer, vi vil tillade, at der indgår i reference-
klassen. Hvilke træk ved dig, kan man det etisk set forsvare at diskriminere på baggrund
af, når du skal have en livsforsikring, ansøger om et job eller søger ind på en uddannelse? 630
F or at svare på det spørgsmål bliver vi for det tredje nødt til at inddrage begrebet ret-
færdighed.

------------

søger, hvor god algoritmen er til at komme med korrekte forudsigelser, er der paradoksalt
nok ingen forskel på farvede og hvide; blandt de, der blev bedømt som højrisiko, var pro-
centdelen af farvede, der var blevet fejlplaceret, nogenlunde lige så stor som procentdelen 605
af hvide, og tilsvarende for de, der blev vurderet som højrisiko. Det betyder med andre
ord, at risikoscoren er nogenlunde lige pålidelig, uanset hvilken etnicitet den person, den
bliver lavet for, har. Y dermere har det vist sig, at man ikke kan have lighed for begge mål
samtidig. Enten vil algoritmen som nu være lige pålidelig i sine forudsigelser uanset etnici-
tet, men til gengæld have en systematisk skævhed i forhold til enkeltpersoners risiko for at 610
blive fejlplaceret, eller også kan man gøre risikoen for at blive fejlplaceret lige stor uanset
etnicitet, hvilket til gengæld betyder, at algoritmen vil være mindre pålidelig for den ene
etniske gruppe end for den anden (for en stringent matematiske gennemgang, se Kleinberg,
Mullainathan og Raghavan, 2017 ). Det er selvfølgelig en interessant etisk udfordring!
Når man skal diskutere hvorvidt brugen af prædiktive algoritmer kan forsvares etisk er 615
der er en række ting, det kan være godt at få på plads.

## Generation

In this step, we will generate an answer to the question using the retrieved documents as context. We will use the OpenAI API to generate the answer.


In [42]:
prompt = PromptTemplate("""You are a helpful assistant that answers questions about the US tv show known as "The Office" using provided context. 

Question: {query}

Context: 

-----------------------------------
{context}

-----------------------------------

""")

prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Context information is below.
    ---------------------
    {context}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: {query}
    Answer: 
    """,
)


message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Context information is below.
    ---------------------
    En
algoritme som COMP AS, der har en større tilbøjelighed til at holde farvede i fængsel end
ikke-farvede, vil bidrage til fastholde denne strukturelle ulighed. En datadreven algoritme
afspejler virkeligheden, som den er, men hvis virkeligheden er racistisk, vil algoritmen også
blive det, og hvis algoritmen bliver brugt uden forbehold, kan den bidrage til at fastholde 685
historisk betingede diskriminerende strukturer. Der er derimod ikke noget, der tyder på, at
mænds kortere levealder er et udtryk for strukturel diskrimination. Så med andre ord er
det vigtigt, at se på den kontekst, en algoritme skal indgå i, når man overvejer, om den er
retfærdig.
Pandoras black box 690
I en traditionel statistisk model vil man have nogenlunde styr på de variable, modellen
inddrager. Det er dog værd at bemærke, at tingene selv med traditionelle algoritmer ik-
ke altid er så klare; COMP AS-algoritmen inddrager således ikke direkte etnicitet som en
variabel. Den inddrager til gengæld flere variable, der er tæt korrelerede med etnicitet, og
dermed indgår etnicitet indirekte i algoritmen. Det gør naturligvis tingene mere besværlige, 695
16Henrik Kragh Sørensen og Mikkel Willum Johansen (apr. 2022). „Invitation til de datalogiske fags
videnskabsteori“ . Lærebog til brug for undervisning ved Institut for Naturfagenes Didaktik, Københavns
Universitet. Under udarbejdelse.
Kapitel 10, version 151 (2021-06-05).

Algoritmen til forudsigelse af frafald inddrog således direkte elevernes etnicitet
som en faktor i beregningen, men hvad nu hvis algoritmen havde en bias så den systematisk
gav elever med en etnicitet en højre risikovurdering end elever med en anden?
Spørgsmålet om diskrimination i algoritmer er navnligt blevet diskuteret i forbindelse 595
med den såkaldte COMP AS-algoritme, der bruges i det amerikanske retsvæsen til vurdering
af kriminelles tilbagefaldsrisiko. En gennemgang af algoritmen viste imidlertid en systema-
tisk skævhed i de fejl, algoritmen lavede. Således havde farvede, der ikke senere begik ny
kriminalitet, en langt højere chance for fejlagtigt at få en høj risikoscore end hvide, og hvide
havde omvendt en højere chance for fejlagtigt at få en lav score end farvede (Angwin m.fl., 600
2016 ).
Hvis man imidlertid tager udgangspunkt i de to kategorier høj- og lav-risiko, og under-
14Henrik Kragh Sørensen og Mikkel Willum Johansen (apr. 2022). „Invitation til de datalogiske fags
videnskabsteori“ . Lærebog til brug for undervisning ved Institut for Naturfagenes Didaktik, Københavns
Universitet. Under udarbejdelse.
Kapitel 10, version 151 (2021-06-05).

Det er ret ukontroversielt. Det er straks mere kontroversielt, at man 575
bl.a. i det amerikanske retsvæsen siden starten af den 20. århundrede har brugt lignende
beregninger til at vurdere kriminelles risiko for at begå ny kriminalitet – og at den type
beregninger er blevet brugt i vurderingen strafudmåling, prøveløsladelser mv. (Carlson, A.
(2017). The Need for T ransparency in the Age of Predictive Sentencing Algorithms. Iowa
Law Review, 103(1), 303–329.). I de sidste årtier har lettere adgang til store datamængder 580
og fremkomsten af diverse machine learning teknikker gjort det lettere at udvikle den type
systemer og diverse former for prædiktive algoritmer bruges i dag på en lang række områder
fra reklamer til sortering af jobansøgninger.
Det er dog ikke helt uproblematisk at bruge prædiktive algoritmer. Som vi så tidligere, er
der en række epistemiske problemer forbundet med machine learning. Der er typisk en meget 585
direkte sammenhæng mellem epistemiske og etiske overvejelser; hvis de etiske overvejelser
inddrager utilititaristiske komponenter er det i høj grad væsentligt at forstå, hvor godt en
given algoritme i praksis virker. Ville den etiske vurdering af algoritmen til forudsigelse af
frafald i gymnasiet fx falde anderledes ud, hvis systemet kun havde en træfsikkerhed på
67%? Der er imidlertid også et anden og vanskeligere etisk aspekt specielt hvis man bruger 590
algoritmiske forudsigelser til at træffe afgørelser, der har betydning for enkeltindividers
muligheder.

F or det første er de etiske argumenter
for at bruge prædiktive algoritmer typisk utilitaristiske; den grundlæggende påstand er, at
en algoritme som COMP AS på den eller anden måde øger den samlede nytte i samfundet og
det er på den baggrund algoritmen skal vurderes (der kan selvfølgelig også være ikke-etiske
begrundelser som kommercielle interesser eller andet). 620
F or andet skal være klar over, at prædiktive algoritmer altid diskriminerer, i den forstand
at din fremtidige adfærd estimeres på baggrund af den fortidige adfærd af den gruppe, som
du anses for at tilhøre. Hvis man fx vil estimere hvor længe en 25 årig, ikke-rygende, køben-
havnsk studerende lever, vil man se på, hvor længe folk i en referenceklasse af personer, der
minder om vores 25-årige studerende i gennemsnit har levet. Det er ganske simpelt sådan, 625
statistiske forudsigelser fungerer (i det mindste i den frekventistisk tolkning af sandsynlig-
hed, som er mest naturlig i denne sammenhæng). Spørgsmålet er altså ikke om algoritmerne
må diskriminere, men i højere grad hvilke faktorer, vi vil tillade, at der indgår i reference-
klassen. Hvilke træk ved dig, kan man det etisk set forsvare at diskriminere på baggrund
af, når du skal have en livsforsikring, ansøger om et job eller søger ind på en uddannelse? 630
F or at svare på det spørgsmål bliver vi for det tredje nødt til at inddrage begrebet ret-
færdighed.

søger, hvor god algoritmen er til at komme med korrekte forudsigelser, er der paradoksalt
nok ingen forskel på farvede og hvide; blandt de, der blev bedømt som højrisiko, var pro-
centdelen af farvede, der var blevet fejlplaceret, nogenlunde lige så stor som procentdelen 605
af hvide, og tilsvarende for de, der blev vurderet som højrisiko. Det betyder med andre
ord, at risikoscoren er nogenlunde lige pålidelig, uanset hvilken etnicitet den person, den
bliver lavet for, har. Y dermere har det vist sig, at man ikke kan have lighed for begge mål
samtidig. Enten vil algoritmen som nu være lige pålidelig i sine forudsigelser uanset etnici-
tet, men til gengæld have en systematisk skævhed i forhold til enkeltpersoners risiko for at 610
blive fejlplaceret, eller også kan man gøre risikoen for at blive fejlplaceret lige stor uanset
etnicitet, hvilket til gengæld betyder, at algoritmen vil være mindre pålidelig for den ene
etniske gruppe end for den anden (for en stringent matematiske gennemgang, se Kleinberg,
Mullainathan og Raghavan, 2017 ). Det er selvfølgelig en interessant etisk udfordring!
Når man skal diskutere hvorvidt brugen af prædiktive algoritmer kan forsvares etisk er 615
der er en række ting, det kan være godt at få på plads.
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: Redegør for hovedtrækkene i sagen om COMPAS-algoritmen som den beskrives i Angwin et al. (2016).
    Answer: 
    

In [43]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": query}],
    model="gpt-4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Angwin et al. (2016) præsenterer i deres artikel "Machine Bias" en kritisk undersøgelse af COMPAS-algoritmen (Correctional Offender Management Profiling for Alternative Sanctions), som bliver anvendt i det amerikanske retssystem. Denne algoritme bliver brugt til at vurdere risikoen for, at en mistænkt, arrestant eller indsat i fængslet vil begå kriminalitet i fremtiden.

Artiklen påpeger, at trods COMPAS-algoritmens popularitet og udbredte anvendelse, viser deres analyse alvorlige skævheder i dens funktion. I deres data fandt de, at algoritmen indikerer, at sorte amerikanere har en højere risiko for recidiv end hvide amerikanere. Men resultaterne fra deres analyse viser, at sorte personer, der blev vurderet til at være i høj risiko for at begå kriminalitet igen, kun gjorde det i 61% af tilfældene, mens det for hvide personer var 67%. Samtidig viste det sig, at personer, der blev vurderet til lav risiko for recidiv, men alligevel begik en forbrydelse igen, var overvejende hvide.

Disse resultater fik Angwin et al. til at konkludere, at COMPAS-algoritmen er biased og diskriminerer på baggrund af race. De påpeger, at denne bias ikke kun potentielt bidrager til en uretfærdig behandling af sortefolk i det amerikanske retssystem, men også kan underminere algoritmens fremmeste formål: At forudsige fremtidig kriminel adfærd. For hvis de personer, der faktisk udgør en høj risiko, ikke bliver korrekt identifiseret af algoritmen, så svigter den sin primære funktion.

Sagen om COMPAS-algoritmen udstiller altså problemer i anvendelsen af maskinintelligens i det juridiske system, især med hensyn til spørgsmål om bias, diskrimination og fairness, og åbner op for en bredere diskussion om, hvordan vi anvender (og regulere) algoritmer i samfundsmæssige sammenhænge.

In [44]:
stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": message}],
    model="gpt-4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

COMPAS-algoritmen er en prædiktiv algoritme, der blev brugt i det amerikanske retsvæsen til at vurdere risikoen for, at kriminelle ville gentage kriminalitet. Angwin et al. (2016) gennemgik algoritmens resultater og fandt en systematisk skævhed i fejlene, den lavede. Dette betyder, at farvede, der ikke senere begik ny kriminalitet, havde en langt højere sandsynlighed for fejlagtigt at få en høj risikoscore end hvide, og hvide, på den anden side, havde en højere sandsynlighed for fejlagtigt at få en lav score end farvede. 

Dette skabte debat om den etiske udfordring ved brug af COMPAS-algoritmen, da dette strukturelt forstærker eksisterende uligheder og diskriminationer i samfundet. Algoritmen var på den ene side nogenlunde lige pålidelig med hensyn til forudsigelser for både hvide og farvede, i det forhold, at procentdelen af farvede og hvide, der var blevet fejlplaceret var nogenlunde den samme. På den anden side var der en systematisk skævhed i forhold til enkeltpersoners risiko for at blive fejlplaceret på grund af deres etnicitet.

## SlideGPT

In [73]:
queries = [
    #"Hvad står P, L, A, C og E for i PLACE-akronymet af John Ziman"
    #"Redegør for hovedtrækkene i sagen om COMPAS-algoritmen som den beskrives i Angwin et al. (2016)."
    #"Hvad er verifikation og falsifikation?"
    "Redegør for hovedtrækkene i sagen om COMPAS-algoritmen som den beskrives i Angwin et al. (2016)."
]

query = queries[-1]

result = collection.query(query_texts=[query], n_results=10)
context = result["documents"][0]
#display(Markdown(f"------------\n\n{"\n\n------------\n\n".join(context)}"))

formatted_text = "\n\n------------\n\n".join(context)

# Display the formatted markdown
display(Markdown(f"{formatted_text}"))

prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Context information is below.
    ---------------------
    {context}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: {query}
    Answer: 
    """,
)

prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Specifically, you will provide your answer such that it is generated as PowerPoint slide(s) content with a MAXIMUM of 3 PowerPoint slides. If helpful, please provide the answer using bullet points.
    Context information is below.
    ---------------------
    {context}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: {query}
    Answer: 
    """,
)

message = prompt.format(query=query, context="\n\n".join(context))
display(Markdown(f"{message}"))

stream = openai_client.chat.completions.create(
    messages=[{"role": "user", "content": message}],
    model="gpt-4",
    stream=True)

output = ""
for chunk in stream:
    if chunk.choices:  # Check if the list is not empty
        output += chunk.choices[0].delta.content or ""
    display(Markdown(f"{output}"), clear=True)

Slide 1: COMPAS-algoritmens Bias
- Beskyldninger om at COMPAS-algoritmen, som bruges i det amerikanske retsvæsen, gav konsekvent sorte en højere kriminalitets-score.
- Skemaet udfyldt af fanger kan udregne en risikoscore for sandsynligheden for at begå ny kriminalitet.
- Etnicitet kan indgå indirekte i algoritmen, da der er mange variable, der er tæt korrelerede med etnicitet.

Slide 2: Etniske Skævheder i COMPAS-algoritmen
- En gennemgang af COMPAS-algoritmen afslørede en systematisk skævhed i de fejl, algoritmen lavede.
- Farvede, der ikke senere begik ny kriminalitet, havde en langt højere chance for fejlagtigt at få en høj risikoscore end hvide. 
- Hvide havde omvendt en højere chance for fejlagtigt at få en lav score end farvede.

Slide 3: Udfordring med COMPAS-algoritmen
- Algoritmerne kan være svære at adressere da de ofte fremstår som neutrale og objektive.
- Udfordringen er: enten vil algoritmen være lige pålidelig i sine forudsigelser uanset etnicitet men vil have en systematisk skævhed i forhold til enkeltpersoners risiko for at blive fejlplaceret, eller man kan gøre risikoen for at blive fejlplaceret lige stor uanset etnicitet, hvilket vil gøre at algoritmen vil være mindre pålidelig for den ene etniske gruppe end for den anden.

In [71]:
display(Markdown(f"{message}"))

You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Context information is below.
    ---------------------
    Enhver videnskabelig teori skal i princippet kunne falsiﬁceres
Henrik Kragh Sørensen (IND, KU) Vi lærer af at tage fejl VtDat •Uge 1 3/12
Poppers videnskabelige metode
1Det er let at opnå bekræftelser eller veriﬁkationer for næsten enhver
teori—hvis man søger efter bekræftelser.
2Bekræftelser bør kun tælle, hvis de er resultatet af dristige
forudsigelser ; dvs. hvis vi uden at være ledet af den pågældende teori
ville have forventet en begivenhed, som var uforenelig med teorien—
en begivenhed som ville have gendrevet teorien.
3Enhver „god“ videnskabelig teori er et forbud: den forbyder, at visse
ting sker. Jo mere en teori forbyder, jo bedre er den.
4En teori som ikke kan gendrives af nogen tænkelig begivenhed er
ikke-videnskabelig. Det er ikke som nogle mennesker tror en dyd, men
derimod en last hos en teori, at den ikke kan gendrives.
5Enhver virkelig afprøvning af en teori er et forsøg på at falsiﬁcere den
eller gendrive den. Prøvbarhed er det samme som falsiﬁcerbarhed; men
der er grader af prøvbarhed: nogle teorier er mere prøvbare, mere
udsatte for gendrivelse end andre; man kunne sige, at de løber en
større risiko.
6Bekræftende vidnesbyrd bør ikke tælle undtagen når de er resultater af
en virkelig afprøvning af teorien ; og dette betyder, at de kan fremstilles
som et alvorligt, men mislykket forsøg på at falsiﬁcere teorien.

om formel verifikation af software og hardware, og det juridiske begreb, der i USA er baseret
på ’beyond reasonable doubt’ (se også MacKenzie, 2004 ). Retssagerne endte i stedet med, at 615
der blev indgået hemnmelige forlig med tavshedsklausuler, som gør, at vores viden stadig
er begrænset. Men man kan godt antage, at T oyota så selv betragtelige forlig som en
mere kosteffektiv løsning på sagen end omfattende og højtprofilerede tilbagetrækninger af
modeller fra hele verden (Gokhale, Brooks og T remblay , 2014 ).
Men der er også et datalogisk perspektiv at antage, som handler om, hvordan man sik- 620
rer sine kritiske systemer mod uforudsete fejl. Og her er der to oplagte muligheder: Vi kan
enten forsøge at bevise korrektheden af vores kritiske systemer igennenm formel verifikation.
Men det er oftest meget omkostningsfuldt, og vi løber alligevel ind i de begrænsninger, som
vi allerede har diskuteret i forhold til overgangen fra lukkede til åbne systemer. Eller vi
kan forsøge at underkaste vores kritiske systemer omfattende, systematisk og teoridreven 625
afprøvning. Og her når vi så, trods transportministerens naive logiske fejlslutning, til er-
kendelse af, at vi ikke kan være sikre på, at vi har fanget alle potentielle fejl og problemer.
Derfor må vi gå en mellemvej, som vi skal se nærmere på i kapitel ??

(I
sådanne tilfælde taler jeg om „befæstende vidnesbyrd“.)
7Nogle virkeligt prøvbare teorier hyldes stadig af deres beundrere, når
man har fundet ud af, at de er falske—f.eks. ved ad hocat indføre en
eller anden hjælpeantagelse eller ved ganske at omfortolke teorien ad
hocpå en sådan måde, at den undgår gendrivelse. Sådanne
fremgangsmåder er altid mulige, men de redder kun teorien fra
gendrivelse ved at ødelægge eller i det mindste sænke dens
videnskabelige status.
K. Popper (1998). „Science. Conjectures and Refutations“. I: Philosophy of Science. The
Central Issues . Udg. af M. Curd og J. A. Cover. New York & London: W. W. Norton &
Company, ss. 3–10, s. 7.Henrik Kragh Sørensen (IND, KU) Vi lærer af at tage fejl VtDat •Uge 1 4/12

Nogenlunde samtidig med disse store skandaler blev man klar over, at direkte uredelig-
hed måske slet ikke var det største problem i forhold til videnskabens integritet. En række
store spørgeskemaundersøgelser af forskeres adfærd viste, at alvorlige overskridelser som
plagiering og fabrikation af data kun forekommer sjældent (eller i hvert fald indrømmede 825
kun få forskere i undersøgelserne at de havde gjort den slags), men til gengæld var mindre
overskridelser af normerne forbavsende udbredte. I den mest berømte af undersøgelserne,
der involverede mere end 3000 amerikanske forskere, indrømmede kun 0,3% af forskerne,
at de havde forfalsket eller manipuleret med data, men 15,5% svarede, at de havde ændret
på en undersøgelses metode eller resultat efter pres fra en sponsor, 15,3% svarede, at de 830
19Henrik Kragh Sørensen og Mikkel Willum Johansen (apr. 2022). „Invitation til de datalogiske fags
videnskabsteori“ . Lærebog til brug for undervisning ved Institut for Naturfagenes Didaktik, Københavns
Universitet. Under udarbejdelse.
Kapitel 10, version 151 (2021-06-05).

Quiz uge 4: Hvad er ”datalogiens teoretiske paradigme”

Quiz uge 4: Hvad er ”datalogiens teoretiske paradigme”

Quiz uge 4: Hvad er ”datalogiens teoretiske paradigme”

Modellers nytte
Hvis modeller ikke giver sandhed, hvad skal vi så med dem? De er
heuristiske værktøjer til videre forskning. Selvom de er ﬁktive, kan de
„resonate with nature“ (være i harmoni med virkeligheden), siger
Cartwright. Og de kan bruges til at ﬁnde fejl i vores intuitioner.
N. Oreskes, K. Shrader-Frechette og K. Belitz (1994). „Veriﬁcation, Validation, and Con-
ﬁrmation of Numerical Models in the Earth Sciences“. Science. Ny rk., bd. 263, nr. 5147,
ss. 641–646. url: http://www.jstor.org/stable/2883078 .
Henrik K. Sørensen (IND, KU) Matematiske modeller VtDat •Uge 5 23/25
Når modeller skaber verden
Henrik K. Sørensen (IND, KU) Matematiske modeller VtDat •Uge 5 24/25

Udvalgene og de bekendtgørelser og lovtek-
ster, der sætter rammerne for dem, har udviklet sig en del siden da, og udviklingen er et
godt eksempel på, hvordan praksis, etik og jura gensidigt påvirker hinanden. Udvalgene
blev i 2017 omdannet til et nævn, Nævnet for Videnskabelig Uredelighed (NVU), og defi-
nitionen af uredelighed blev ved samme lejlighed væsentligt revideret. I den nuværende lov 850
beskriver man videnskabelig uredelighed som: ’F abrikering, forfalskning og plagiering, som
er begået forsætligt eller groft uagtsomt ved planlægning, gennemførelse eller rapportering
af forskning’ ( Lov om videnskabelig uredelighed m.v. 2017 , §3.1), hvilket svarer til den ty-
piske internationale definition, hvor uredelighed netop forstås som fabrikering, forfalskning
og plagiering (FFP). I den danske lov defineres de tre centrale begreber som: 855
F abrikering: Uoplyst konstruktion af data eller substitution med fiktive data.
F orfalskning: Manipulation af forskningsmateriale, udstyr eller processer samt
ændring eller udeladelse af data eller resultater, hvorved forskning fremstår mis-
visende.
Plagiering: Tilegnelse af andres ideer, processer, resultater, tekst eller særlige 860
begreber uden retmæssig kreditering ( Lov om videnskabelig uredelighed m.v.
2017 , §3.2–3.4).

En af de vigtigste reaktioner på den logiske positivisme blev formuleret af filosoffen Karl 435
Popper (1902–1994), der også stod bag en del af kritikken af den tidligere position. Centralt
for Popper stod, at det ligesom var ’for nemt’ at producere sande videnskabelige udsagn,
hvis de kun skulle omfatte de kendte iagttagelser (Popper, 1996 ). Hele den videnskabelige
proces burde, mente han, være meget mere modig og ekspansiv: Det galdt om at forstå
mere, og derfor måtte man tage mere fejl, for vi lærer igennem vores fejl, ifølge Popper . 440
Popper formulerede derfor en videnskabelig metode baseret på falsifikation i stedet
for de logiske positivisters verifikationisme : Pointen skulle være at formulere interessante
udsagn og ihærdigt forsøge at modbevise (gendrive, falsificere) dem.
F or at denne proces skal kunne virke, må man først igennem en heuristisk fase, hvor
forskellige muligheder åbent afsøges, inden man kan få en god ide om, hvad man ønsker at 445
undersøge (se figur 5). Derefter følger en foreløbig undersøgelse, hvor der udføres nogle forsøg
med henblik på at opstille en teori eller hypotese , hvis gyldighed rækker ud over de allerede
foretagne forsøg. Denne heuristiske del, frem til formuleringen af den første hypotese, er
ikke noget, Popper beskrev i stor detaljer, og man har senere diskuteret, hvori denne del
12Henrik Kragh Sørensen og Mikkel Willum Johansen (apr. 2022). „Invitation til de datalogiske fags
videnskabsteori“ .
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: Hvad er verifikation og falsifikation?
    Answer: 
    

## Normal example using LlamaIndex

In this example, we will use LlamaIndex to abstract the indexing and retrieval steps. This shows how easily the same pipeline can be implemented using LlamaIndex.


In [ ]:

#%pip install llama-index-embeddings-azure-openai
#%pip install llama-index-llms-azure-openai

In [12]:
import chromadb
from chromadb import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# ChromaDB Vector Store
chroma_client = chromadb.PersistentClient(
    path="./data/baseline-rag-pdf-docs/chromadb", settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.get_or_create_collection(
    name="VtDat", metadata={"hnsw:space": "cosine"})
vector_store = ChromaVectorStore(chroma_collection=collection)


llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt-4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Define the ingestion pipeline to add documents to vector store
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=20),
        embed_model,
    ],
    vector_store=vector_store,
)

# Create index with the vector store and using the embedding model
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model)

In [13]:
# Fetch documents
documents = SimpleDirectoryReader('./data/docs').load_data()

# Run pipeline
pipeline.run(documents=documents)

print("Indexing complete")

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 65 0 (offset 0)
Ignoring wrong pointing object 96 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)


Indexing complete


In [46]:
index.as_retriever()

#### Create base QueryEngine from LlamaIndex


In [26]:
query_engine = index.as_query_engine(llm=llm, verbose=True)

#### Or alternatively, create a CustomQueryEngine


In [20]:
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.response_synthesizers import BaseSynthesizer


class MyRetriever(BaseRetriever):
    def retrieve(self, query_str: str, max_documents: int = 10):
        # Get all documents relevant to the query
        all_documents = super().retrieve(query_str)

        # Return only the first `max_documents` documents
        return all_documents[:max_documents]


qa_prompt = PromptTemplate(
    """You are a helpful assistant that answers questions about the course material from "Philosophy of Computer Science (VtDat)" using provided context.
    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, answer the query. Always provide an answer in the Danish language.
    Query: {query_str}
    Answer: 
    """,
)


class RAGQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    retriever: MyRetriever #BaseRetriever
    response_synthesizer: BaseSynthesizer
    llm: OpenAI
    qa_prompt: PromptTemplate

    def custom_query(self, query_str: str):
        #nodes = self.retriever.retrieve(query_str)#
        nodes = self.retriever.retrieve(query_str, max_documents=5)
        context_str = "\n\n".join([n.node.get_content() for n in nodes])
        print("Prompt:\n\n", qa_prompt.format(
            context_str=context_str, query_str=query_str))
        response = self.llm.complete(
            qa_prompt.format(context_str=context_str, query_str=query_str)
        )

        return str(response)


synthesizer = get_response_synthesizer(response_mode="compact")
query_engine = RAGQueryEngine(
    retriever = index.as_retriever(),
    response_synthesizer=synthesizer,
    llm=llm,
    qa_prompt=qa_prompt,
)

TypeError: Can't instantiate abstract class MyRetriever with abstract method _retrieve

In [45]:
queries = [
    #"What are some of the made-up characters impersonated by Michael Scott? Give a oneliner description for each character.",
    #"Who is the character that is known for his 'That's what she said' jokes in The Office?",
    #"Which character loves cats?",
    #"A character has a heart attack in an episode of the show. Who is it?",
    #"What character lives proudly on a farm?",
    #"What happens to Kevin's Famous Chili when he brings it to the office?",
    #"Why did Michael Scott play the Savannah murder game?",
    #"How many marriages are in the show?",
    "Hvad dækker P, L, A, C og E over i PLACE-akronymet af John Ziman over?"
]

query = queries[-1]

response = query_engine.query(query)
display(Markdown(f"{response}"))

InvalidCollectionException: Collection e3406e00-da5e-41a3-b052-410e1e72d3a9 does not exist.

## Simplest RAG implementation using LlamaIndex


In [4]:
llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="gpt-4",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"), # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference?WT.mc_id=AZ-MVP-5004796
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Fetch documents
documents = SimpleDirectoryReader('./data/baseline-rag-pdf-docs').load_data()

from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

# build VectorStoreIndex that takes care of chunking documents
# and encoding chunks to embeddings for future retrieval
index = VectorStoreIndex.from_documents(documents=documents)#, 
                                        #api_key=os.environ['OPENAI_API_KEY'],
                                        #base_url=os.environ['AZURE_API_BASE'],
                                        #app_url=os.environ['AZURE_OPENAI_ENDPOINT'])
#index = VectorStoreIndex.from_documents(documents=documents, embed_model=embed_model, llm=llm, verbose=True)

# The QueryEngine class is equipped with the generator
# and facilitates the retrieval and generation steps
query_engine = index.as_query_engine()

# Use your Default RAG
response = query_engine.query(query)
display(Markdown(f"{response}"))

TypeError: AzureOpenAI.__init__() got an unexpected keyword argument 'model'

In [37]:

from ironpdf import *

# Load existing PDF document
pdf = PdfDocument.FromFile("C:/Users/jach/Documents/JacobsDocs/KU/VtDat/Tekster/Uge3/Kuhn_objektivitet.pdf")

# Extract text from specific page in the document
page_text = pdf.ExtractTextFromPage(1)


LicensingException: IronPDF must be licensed for development.
Please receive a free trial key instantly to your email by visiting:
https://ironpdf.com/licensing/?utm_source=product
   at IronPdf.License.kpbbda()
   at IronPdf.PdfDocument.axjqme(IEnumerable`1 ralkye, TextExtractionOrder ralkyf)
   at IronPdf.PdfDocument.ExtractTextFromPages(IEnumerable`1 PageIndices, TextExtractionOrder Order)
   at IronPdf.PdfDocument.ExtractTextFromPage(Int32 PageIndex, TextExtractionOrder Order)
   at IronPdf.PdfDocument.ExtractTextFromPage(Int32 PageIndex)
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeDirectByRefWithFewArgs(Object obj, Span`1 copyOfArgs, BindingFlags invokeAttr)